<a href="https://colab.research.google.com/github/siddharth-kale/Twitter-Network-Analysis/blob/main/Hashtag_Trenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea
!pip install snscrape
import snscrape.modules.twitter as sntwitter
import re
!pip install networkx
import networkx as nx

In [ ]:
pd.options.display.max_rows=1000

In [ ]:
keyword = ''
startdate = "2020-01-01"
enddate = "2021-11-11"

In [ ]:
maxtweets = 5000000
tweetlist=[]
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('{} since:{} until:{}'.format(keyword,startdate,enddate)).get_items()):
   if i>maxtweets:
         break
   tweetlist.append(tweet)

tweets_df = pd.DataFrame(tweetlist)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
tweets_df.to_csv('/content/gdrive/MyDrive/Projects/Twitter Network Analysis/Data/{}Trender.csv'.format(keyword))

In [ ]:
tweets_df = pd.read_csv('/content/gdrive/MyDrive/Projects/Twitter Network Analysis/Data/{}Trender.csv'.format(keyword))

In [ ]:
tweets_df = tweets_df.drop(columns=['Unnamed: 0'])
tweets_df = tweets_df.drop_duplicates(subset='id')
tweets_df

In [ ]:
interaction_df = pd.DataFrame(columns=['source','target'])
ind=0
for i in range(0,len(tweets_df)):
  templist=[]
  templist=re.findall('@\w+',str(tweets_df.iloc[i,2]))
  
  if len(templist)==0:
    interaction_df.loc[ind,'source'] = tweets_df.iloc[i,4]
    interaction_df.loc[ind,'target'] = ''
    ind = ind+1

  else:
    for j in range(0,len(templist)): 
      interaction_df.loc[ind,'source'] = tweets_df.iloc[i,4]
      interaction_df.loc[ind,'target'] = templist[j][1:]
      ind = ind+1

interaction_df.head(999)



In [ ]:
temp=pd.DataFrame(interaction_df.stack()).reset_index().drop(columns=['level_0'])
temp=temp[temp['level_1']=='source']
temp.columns=['type','source']
temp

In [ ]:
htrenders=temp['source'].value_counts().to_frame().reset_index()
htrenders.columns=['Account','Tweets']
htrenders.head(10)

In [ ]:
nodes_df = pd.DataFrame(interaction_df.stack()).reset_index().drop(columns=['level_0','level_1'])
nodes_df.columns=['username']
complete_nodes=list(set(nodes_df[nodes_df['username']!='']['username'].to_list()))

In [ ]:
G = nx.DiGraph()

In [ ]:
for i in range(0,len(complete_nodes)):
  G.add_node(complete_nodes[i])

print(G.number_of_nodes())

In [ ]:
full_interaction_df = interaction_df[interaction_df['target']!='']
for i in range(0,len(full_interaction_df)):
  G.add_edge(full_interaction_df.iloc[i,0],full_interaction_df.iloc[i,1])

In [ ]:
nodedegree=pd.DataFrame.from_dict(dict(G.out_degree),orient='index')
nodedegree = nodedegree.reset_index()
nodedegree.columns=['Name','Outreach']
nodedegree = nodedegree.sort_values(by='Outreach',ascending=False)
nodedegree.head(50)

In [ ]:
d =dict(G.out_degree)
plt.figure(figsize=(400,400))
plt.title('Twitter Interaction Graph for {}'.format(keyword))
nx.draw(G,node_size=[(x+1) * 100 for x in list(d.values())],with_labels=True)
plt.savefig('{}.png'.format(keyword))